![pageindex_banner](https://pageindex.ai/static/images/pageindex_banner.jpg)

<p align="center"><i>Reasoning-based RAG&nbsp; ✧ &nbsp;No Vector DB&nbsp; ✧ &nbsp;No Chunking&nbsp; ✧ &nbsp;Human-like Retrieval</i></p>

<p align="center">
  <a href="https://vectify.ai">🏠 Homepage</a>&nbsp; • &nbsp;
  <a href="https://dash.pageindex.ai">🖥️ Dashboard</a>&nbsp; • &nbsp;
  <a href="https://docs.pageindex.ai/quickstart">📚 API Docs</a>&nbsp; • &nbsp;
  <a href="https://github.com/VectifyAI/PageIndex">📦 GitHub</a>&nbsp; • &nbsp;
  <a href="https://discord.com/invite/VuXuf29EUj">💬 Discord</a>&nbsp; • &nbsp;
  <a href="https://ii2abc2jejf.typeform.com/to/tK3AXl8T">✉️ Contact</a>&nbsp;
</p>

<div align="center">

[![Star us on GitHub](https://img.shields.io/github/stars/VectifyAI/PageIndex?style=for-the-badge&logo=github&label=⭐️%20Star%20Us)](https://github.com/VectifyAI/PageIndex) &nbsp;&nbsp; [![Follow us on X](https://img.shields.io/badge/Follow%20Us-000000?style=for-the-badge&logo=x&logoColor=white)](https://twitter.com/VectifyAI)

</div>

---

# Simple Vectorless RAG with PageIndex

## PageIndex Introduction
PageIndex is a new **reasoning-based**, **vectorless RAG** framework that performs retrieval in two steps:  
1. Generate a tree structure index of documents  
2. Perform reasoning-based retrieval through tree search  

<div align="center">
  <img src="https://docs.pageindex.ai/images/cookbook/vectorless-rag.png" width="70%">
</div>

Compared to traditional vector-based RAG, PageIndex features:
- **No Vectors Needed**: Uses document structure and LLM reasoning for retrieval.
- **No Chunking Needed**: Documents are organized into natural sections rather than artificial chunks.
- **Human-like Retrieval**: Simulates how human experts navigate and extract knowledge from complex documents.
- **Transparent Retrieval Process**: Retrieval based on reasoning — say goodbye to approximate semantic search ("vibe retrieval").

## 📝 Notebook Overview

This notebook demonstrates a simple, minimal example of **vectorless RAG** with PageIndex. You will learn how to:
- [x] Build a PageIndex tree structure of a document
- [x] Perform reasoning-based retrieval with tree search
- [x] Generate answers based on the retrieved context

> ⚡ Note: This is a **minimal example** to illustrate PageIndex's core philosophy and idea, not its full capabilities. More advanced examples are coming soon.

---

## Step 0: Preparation



#### 0.1 Install PageIndex

In [1]:
%pip install -q --upgrade pageindex

#### 0.2 Setup PageIndex

In [9]:
from pageindex import PageIndexClient
import pageindex.utils as utils

# Get your PageIndex API key from https://dash.pageindex.ai/api-keys
PAGEINDEX_API_KEY = "25e12aac29db4cbdb117670f58a44ecb"
pi_client = PageIndexClient(api_key=PAGEINDEX_API_KEY)

#### 0.3 Setup LLM

Choose your preferred LLM for reasoning-based retrieval. In this example, we use OpenAI’s GPT-4.1.

In [10]:
# import openai
# OPENAI_API_KEY = ""

# async def call_llm(prompt, model="gpt-5.2", temperature=0):
#     client = openai.AsyncOpenAI(api_key=OPENAI_API_KEY)
#     response = await client.chat.completions.create(
#         model=model,
#         messages=[{"role": "user", "content": prompt}],
#         temperature=temperature
#     )
#     return response.choices[0].message.content.strip()

In [39]:
pip install google-generativeai

In [40]:
pip install -q -U google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.9/721.9 kB 29.2 MB/s eta 0:00:00


In [54]:
import os
import asyncio
from google import genai
from google.colab import userdata
# os.environ["GEMINI_API_KEY"] = "xxxxxx"
os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")
_client = genai.Client()

def _call_gemini_sync(prompt: str, model: str, temperature: float) -> str:
    response = _client.models.generate_content(
        model=model,
        contents=prompt,
        config={
            "temperature": temperature,
        },
    )
    if not response.text:
        raise RuntimeError("Gemini returned empty response")
    return response.text.strip()

async def call_llm(
    prompt: str,
    model: str = "gemini-3-flash-preview",
    temperature: float = 0,
) -> str:
    loop = asyncio.get_running_loop()
    return await loop.run_in_executor(
        None, _call_gemini_sync, prompt, model, temperature
    )


## Step 1: PageIndex Tree Generation

#### 1.1 Submit a document for generating PageIndex tree

In [13]:
import os, requests

In [34]:
pdf_url = "https://arxiv.org/pdf/2507.17061.pdf"
pdf_path = os.path.join("../data", pdf_url.split('/')[-1])
os.makedirs(os.path.dirname(pdf_path), exist_ok=True)

In [35]:
def safe_path(url: str, save_dir: str, ext: str) -> str:
    import os

    name = url.split("/")[-1]
    if not name.endswith(ext):
        name += ext

    path = os.path.join(save_dir, name)
    os.makedirs(save_dir, exist_ok=True)
    return path

In [29]:
path = safe_path(pdf_url, "../data", ".pdf")

In [36]:
response = requests.get(pdf_url)
with open(path, "wb") as f:
    f.write(response.content)
print(f"Downloaded {pdf_url}")

Downloaded https://arxiv.org/pdf/2507.17061.pdf


In [31]:
print(response.headers.get("Content-Type"))

application/pdf


In [32]:
doc_id = pi_client.submit_document(pdf_path)["doc_id"]
print('Document Submitted:', doc_id)

PageIndexAPIError: Failed to submit document: {"detail":"Only PDF files are supported."}

In [23]:
# import os, requests

# # You can also use our GitHub repo to generate PageIndex tree
# # https://github.com/VectifyAI/PageIndex

# pdf_url = "https://arxiv.org/pdf/2507.17061"
# pdf_path = os.path.join("../data", pdf_url.split('/')[-1])
# os.makedirs(os.path.dirname(pdf_path), exist_ok=True)

# response = requests.get(pdf_url)
# with open(pdf_path, "wb") as f:
#     f.write(response.content)
# print(f"Downloaded {pdf_url}")

# doc_id = pi_client.submit_document(pdf_path)["doc_id"]
# print('Document Submitted:', doc_id)

#### 1.2 Get the generated PageIndex tree structure

In [37]:
if pi_client.is_retrieval_ready(doc_id):
    tree = pi_client.get_tree(doc_id, node_summary=True)['result']
    print('Simplified Tree Structure of the Document:')
    utils.print_tree(tree)
else:
    print("Processing document, please try again later...")

Simplified Tree Structure of the Document:
[{'title': 'Parallelism Meets Adaptiveness: Scalable...',
  'node_id': '0000',
  'prefix_summary': '# Parallelism Meets Adaptiveness: Scalab...',
  'nodes': [{'title': 'Abstract',
             'node_id': '0001',
             'summary': 'This paper introduces a novel multi-agen...'},
            {'title': '1 Introduction',
             'node_id': '0002',
             'summary': 'This text introduces a new framework for...'},
            {'title': '2 Related Work',
             'node_id': '0003',
             'summary': 'This text reviews related work in multi-...'},
            {'title': '3 Problem Statement and Motivating Use C...',
             'node_id': '0004',
             'summary': 'The text discusses the limitations of cu...'},
            {'title': '4 Core Innovations in Adaptive Coordinat...',
             'node_id': '0005',
             'summary': 'The text discusses the inadequacy of sta...'},
            {'title': '5 System Archite

## Step 2: Reasoning-Based Retrieval with Tree Search

#### 2.1 Use LLM for tree search and identify nodes that might contain relevant context

In [55]:
import json

query = "What are the conclusions in this document?"

tree_without_text = utils.remove_fields(tree.copy(), fields=['text'])

search_prompt = f"""
You are given a question and a tree structure of a document.
Each node contains a node id, node title, and a corresponding summary.
Your task is to find all nodes that are likely to contain the answer to the question.

Question: {query}

Document tree structure:
{json.dumps(tree_without_text, indent=2)}

Please reply in the following JSON format:
{{
    "thinking": "<Your thinking process on which nodes are relevant to the question>",
    "node_list": ["node_id_1", "node_id_2", ..., "node_id_n"]
}}
Directly return the final JSON structure. Do not output anything else.
"""

tree_search_result = await call_llm(search_prompt)

#### 2.2 Print retrieved nodes and reasoning process

In [56]:
node_map = utils.create_node_mapping(tree)
tree_search_result_json = json.loads(tree_search_result)

print('Reasoning Process:')
utils.print_wrapped(tree_search_result_json['thinking'])

print('\nRetrieved Nodes:')
for node_id in tree_search_result_json["node_list"]:
    node = node_map[node_id]
    print(f"Node ID: {node['node_id']}\t Page: {node['page_index']}\t Title: {node['title']}")

Reasoning Process:
To find the conclusions of the document, I looked for sections that summarize the results, findings,
and final thoughts. The 'Abstract' (0001) provides a high-level summary of the outcomes. 'Key
Findings' (0015) details the specific results from the experiments. The 'Discussion' (0016)
interprets these results and their implications. Finally, 'Conclusion and Future Work' (0017)
explicitly states the final conclusions and future directions. The 'Case Study' (0011) also contains
summary data about the system's performance which forms the basis of the conclusions.

Retrieved Nodes:
Node ID: 0001	 Page: 1	 Title: Abstract
Node ID: 0011	 Page: 4	 Title: 6 Case Study: Adaptive Coordination for 10-K Analysis
Node ID: 0015	 Page: 6	 Title: 6.4 Key Findings
Node ID: 0016	 Page: 6	 Title: 7 Discussion
Node ID: 0017	 Page: 7	 Title: 8 Conclusion and Future Work


## Step 3: Answer Generation

#### 3.1 Extract relevant context from retrieved nodes

In [57]:
node_list = json.loads(tree_search_result)["node_list"]
relevant_content = "\n\n".join(node_map[node_id]["text"] for node_id in node_list)

print('Retrieved Context:\n')
utils.print_wrapped(relevant_content[:1000] + '...')

Retrieved Context:

## Abstract

Large language model (LLM) agents have shown increasing promise for collaborative task completion.
However, existing multi-agent frameworks often rely on static workflows, fixed roles, and limited
inter-agent communication, reducing their effectiveness in open-ended, high-complexity domains. This
paper present a multi-agent coordination framework that improves the accuracy of Large Language
Models (LLMs) in complex financial document analysis. Unlike existing frameworks that rely on static
routing or linear workflows, our approach introduces Parallel Agent Evaluation, a mechanism where
multiple agents compete on high-ambiguity subtasks. A centralized evaluator scores these parallel
outputs based on factuality and coherence to select the optimal result. We evaluate this
architecture on SEC 10-K filings, demonstrating a 27% improvement in compliance accuracy and a 74%
reduction in revision rates compared to standard static baselines. These results validat

#### 3.2 Generate answer based on retrieved context

In [59]:
answer_prompt = f"""
Answer the question based on the context:

Question: {query}
Context: {relevant_content}

You are the AI expert, how do you think this framework?
"""

print('Generated Answer:\n')
answer = await call_llm(answer_prompt)
utils.print_wrapped(answer)

Generated Answer:

Based on the provided document, here are the conclusions reached by the authors, followed by an AI
expert’s perspective on the framework.

### Part 1: Conclusions from the Document

The authors conclude that moving away from static, linear multi-agent workflows toward an **adaptive
and competitive coordination framework** significantly improves the performance of LLMs in complex,
high-stakes domains like financial analysis. Their specific conclusions include:

1.  **Superior Performance Metrics:** The "Full System" (incorporating parallel evaluation) achieved
a **0.94 compliance accuracy** and **0.92 factual coverage**, representing a **27% improvement** in
accuracy over static baselines.
2.  **Reduction in Errors:** The framework effectively severed the "cascade of errors" typical in
LLM chains, resulting in a **74% reduction in revision rates** and a **73% reduction in redundancy
penalties**.
3.  **Value of Structured Competition:** The authors conclude that redund

---

## 🎯 What's Next

This notebook has demonstrated a **basic**, **minimal** example of **reasoning-based**, **vectorless** RAG with PageIndex. The workflow illustrates the core idea:
> *Generating a hierarchical tree structure from a document, reasoning over that tree structure, and extracting relevant context, without relying on a vector database or top-k similarity search*.

While this notebook highlights a minimal workflow, the PageIndex framework is built to support **far more advanced** use cases. In upcoming tutorials, we will introduce:
* **Multi-Node Reasoning with Content Extraction** — Scale tree search to extract and select relevant content from multiple nodes.
* **Multi-Document Search** — Enable reasoning-based navigation across large document collections, extending beyond a single file.
* **Efficient Tree Search** — Improve tree search efficiency for long documents with a large number of nodes.
* **Expert Knowledge Integration and Preference Alignment** — Incorporate user preferences or expert insights by adding knowledge directly into the LLM tree search, without the need for fine-tuning.



## 🔎 Learn More About PageIndex
  <a href="https://vectify.ai">🏠 Homepage</a>&nbsp; • &nbsp;
  <a href="https://dash.pageindex.ai">🖥️ Dashboard</a>&nbsp; • &nbsp;
  <a href="https://docs.pageindex.ai/quickstart">📚 API Docs</a>&nbsp; • &nbsp;
  <a href="https://github.com/VectifyAI/PageIndex">📦 GitHub</a>&nbsp; • &nbsp;
  <a href="https://discord.com/invite/VuXuf29EUj">💬 Discord</a>&nbsp; • &nbsp;
  <a href="https://ii2abc2jejf.typeform.com/to/tK3AXl8T">✉️ Contact</a>

<br>

© 2025 [Vectify AI](https://vectify.ai)